# SK: Agents using Planner

## Outline:

* Using built in SK Skills: HttpSkill, TimeSkill and MathSkill
* Defining your own tools

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

In [2]:
import semantic_kernel as sk
import os
import logging
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('__name__')
kernel=sk.Kernel(log=logger)

api_key = os.environ['OPENAI_API_KEY']
kernel.add_chat_service(
        "chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key)
)

## Built-in SK tools

In [3]:
kernel.import_skill(sk.core_skills.MathSkill(), "MathSkill")
kernel.import_skill(sk.core_skills.TimeSkill(), "TimeSkill")
kernel.import_skill(sk.core_skills.HttpSkill(), "HttpSkill")

{'deleteAsync': <semantic_kernel.orchestration.sk_function.SKFunction at 0x7f7b43462c10>,
 'getAsync': <semantic_kernel.orchestration.sk_function.SKFunction at 0x7f7b43462d00>,
 'postAsync': <semantic_kernel.orchestration.sk_function.SKFunction at 0x7f7b43462e50>,
 'putAsync': <semantic_kernel.orchestration.sk_function.SKFunction at 0x7f7b43462fa0>}

In [4]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [5]:
ask = "What is the sum of 2319 and 86988?"

In [6]:
original_plan = await planner.create_plan_async(ask, kernel)

INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=1891 request_id=a7d21c4447b27c6a2721560febb6d060 response_code=200


In [7]:
print(original_plan.generated_plan)

{
    "input": "",
    "subtasks": [
        {"function": "MathSkill.Add", "args": {"input": 2319, "Amount": 86988}}
    ]
}


In [8]:
results = await planner.execute_plan_async(original_plan, kernel)
print(results)

89307


In [9]:
ask="whats the date today?"
plan2 = await planner.create_plan_async(ask, kernel)
print(plan2.generated_plan)
result2= await planner.execute_plan_async(plan2, kernel)
print(result2)

INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=1605 request_id=587a552498f6d6e0526d6ec2ebc2d222 response_code=200


{
    "input": "whats the date today?",
    "subtasks": [
        {"function": "TimeSkill.today"}
    ]
}
Sunday, 18 June, 2023


In [10]:
# Plans generated are not always accurate. Here is an example
ask = "What book did Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU) write?"

# IMPORTANT: Copy the skills/WriterSkill directory from https://github.com/microsoft/semantic-kerne into current directory
skills_directory = "skills"
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")

plan3 = await planner.create_plan_async(ask, kernel)
print(plan3.generated_plan)
 

INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=4198 request_id=a932aef5f1fe3730e8762add28a5e3ed response_code=200


{
    "input": "Tom M. Mitchell, American computer scientist and Founders University Professor at CMU",
    "subtasks": [
        {"function": "HttpSkill.getAsync", "args": {"input": "https://www.googleapis.com/books/v1/volumes?q=" + "Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University"}},
        {"function": "WriterSkill.TwoSentenceSummary"}
    ]
}


In [33]:
result3= await planner.execute_plan_async(plan3, kernel)
print(result3)

DEBUG:__name__:Rendering string template: Rewrite the given text like it was written in this style or by: {{$style}}. 
MUST RETAIN THE MEANING AND FACTUAL CONTENT AS THE ORIGINAL.


{{$input}}


DEBUG:__name__:Extracting blocks from template: Rewrite the given text like it was written in this style or by: {{$style}}. 
MUST RETAIN THE MEANING AND FACTUAL CONTENT AS THE ORIGINAL.


{{$input}}


DEBUG:__name__:Rendering list of 5 blocks
DEBUG:__name__:Rendered prompt: Rewrite the given text like it was written in this style or by: compact. 
MUST RETAIN THE MEANING AND FACTUAL CONTENT AS THE ORIGINAL.


$HttpSkill.getAsync


DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "user", "content": "Rewrite the given text like it was written in this style or by: compact. \\nMUST RETAIN THE MEANING AND FACTUAL CONTENT AS THE ORIGINAL.\\n\\n\\n$HttpSkill.getAs

INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=2374 request_id=d93d151df84b25d4f517e2af49c0afa6 response_code=200


Hi Tom,

I wanted to reach out and express my admiration for your incredible skills as a writer. Your ability to craft engaging and thought-provoking pieces is truly impressive.

Thank you for sharing your talents with the world.

Best regards,

[Your Name]


## Define your own tool

In [ ]:
# TODO